# **Import Operating System. Install Spark, Set Environment and Import Packages**

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

In [2]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [891 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,386 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.5 kB]
Get:13 http://

In [3]:
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

In [4]:
# Import packages
from pyspark.sql import SparkSession
from pyspark import SparkFiles

In [5]:
# Start a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

# **Data Exploration and find answer for the queries - Using Temporary View**

In [7]:
# Create a temporary view
df.createOrReplaceTempView("home_sales")

# Verify that the temporary view is created by running a simple query
spark.sql("SELECT * FROM home_sales").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [11]:
# 1. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

from pyspark.sql.functions import year, avg, round

# Filter the DataFrame to include only four-bedroom houses
four_bedroom_df = df.filter(df.bedrooms == 4)

# Extract the year from the date column
four_bedroom_df = four_bedroom_df.withColumn("year", year("date"))

# Group by year and calculate the average price
avg_price_df = four_bedroom_df.groupBy("year").agg(round(avg("price"), 2).alias("average_price"))

# Show the result
avg_price_df.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2022|    296363.88|
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
+----+-------------+



In [12]:
# 2. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

# Filter the DataFrame to include only homes with 3 bedrooms and 3 bathrooms
three_bed_bath_df = df.filter((df.bedrooms == 3) & (df.bathrooms == 3))

# Group by the year the home was built and calculate the average price
avg_price_by_year_built_df = three_bed_bath_df.groupBy("date_built").agg(round(avg("price"), 2).alias("average_price"))

# Show the result
avg_price_by_year_built_df.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|    290555.07|
|      2012|    293683.19|
|      2017|    292676.79|
|      2014|    290852.27|
|      2013|    295962.27|
|      2011|    291117.47|
|      2015|     288770.3|
|      2010|    292859.62|
+----------+-------------+



In [13]:
# 3. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

# Filter the DataFrame to include only homes with 3 bedrooms, 3 bathrooms, 2 floors, and at least 2,000 square feet
filtered_df = df.filter((df.bedrooms == 3) & (df.bathrooms == 3) & (df.floors == 2) & (df.sqft_living >= 2000))

# Group by the year the home was built and calculate the average price
avg_price_by_year_built_df = filtered_df.groupBy("date_built").agg(round(avg("price"), 2).alias("average_price"))

# Show the result
avg_price_by_year_built_df.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|     293965.1|
|      2012|    307539.97|
|      2017|    280317.58|
|      2014|    298264.72|
|      2013|    303676.79|
|      2011|    276553.81|
|      2015|    297609.97|
|      2010|    285010.22|
+----------+-------------+



In [14]:
# 4. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

from pyspark.sql.functions import avg, round
import time

# Start the timer
start_time = time.time()

# Group by "view" rating, calculate the average price, round it, and filter the results
avg_price_by_view_df = df.groupBy("view").agg(round(avg("price"), 2).alias("average_price")) \
    .filter("average_price >= 350000") \
    .orderBy("view", ascending=False)

# Show the result
avg_price_by_view_df.show()

# End the timer
end_time = time.time()

# Calculate the run time
run_time = end_time - start_time
print(f"Query run time: {run_time} seconds")


+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

Query run time: 2.1860997676849365 seconds


# **Cache and Uncache**

In [15]:
# Create a temporary view named 'home_sales'
df.createOrReplaceTempView("home_sales")

# Cache the temporary table
spark.catalog.cacheTable("home_sales")

# Verify that the table is cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")


Is 'home_sales' cached? True


In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [17]:
# Define the query to calculate the average price of a home per "view" rating
from pyspark.sql.functions import avg, round
import time

In [18]:
# Run the query on the cached table and measure the runtime
start_time_cached = time.time()
avg_price_by_view_cached_df = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    GROUP BY view
    HAVING average_price >= 350000
    ORDER BY view DESC
""")
avg_price_by_view_cached_df.show()
end_time_cached = time.time()
runtime_cached = end_time_cached - start_time_cached
print(f"Cached query run time: {runtime_cached} seconds")

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

Cached query run time: 3.377251148223877 seconds


In [ ]:
# Uncache the table to measure uncached runtime
spark.catalog.uncacheTable("home_sales")

# Verify that the table is uncached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached after uncache? {is_cached}")

# Run the query on the uncached table and measure the runtime
start_time_uncached = time.time()
avg_price_by_view_uncached_df = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    GROUP BY view
    HAVING average_price >= 350000
    ORDER BY view DESC
""")
avg_price_by_view_uncached_df.show()
end_time_uncached = time.time()
runtime_uncached = end_time_uncached - start_time_uncached
print(f"Uncached query run time: {runtime_uncached} seconds")

# Compare the runtimes
print(f"Runtime difference (uncached - cached): {runtime_uncached - runtime_cached} seconds")

Is 'home_sales' cached after uncache? False
+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

Uncached query run time: 0.6633355617523193 seconds
Runtime difference (uncached - cached): -0.5847699642181396 seconds


In [ ]:
# 7. Cache the the temporary table home_sales.

# Create a temporary view named 'home_sales'
df.createOrReplaceTempView("home_sales")

# Cache the temporary table
spark.catalog.cacheTable("home_sales")

# Verify that the table is cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")


Is 'home_sales' cached? True


In [ ]:
# Run the query on the uncached table and measure the runtime
start_time_uncached = time.time()
avg_price_by_view_uncached_df = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    GROUP BY view
    HAVING average_price >= 350000
    ORDER BY view DESC
""")
avg_price_by_view_uncached_df.show()
end_time_uncached = time.time()
runtime_uncached = end_time_uncached - start_time_uncached
print(f"Uncached query run time: {runtime_uncached} seconds")

# Compare the runtimes
print(f"Runtime difference (uncached - cached): {runtime_uncached - runtime_cached} seconds")

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

Uncached query run time: 1.0340380668640137 seconds
Runtime difference (uncached - cached): -0.2140674591064453 seconds


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Partition by 'date_built' and save as Parquet
output_path = "/path/to/output_directory1"
df.write.partitionBy("date_built").parquet(output_path)

print(f"Data partitioned by 'date_built' and saved to {output_path}")

Data partitioned by 'date_built' and saved to /path/to/output_directory1


In [ ]:
# 11. Read the parquet formatted data.

# Read the Parquet formatted data
parquet_path = "/path/to/output_directory"
parquet_df = spark.read.parquet(parquet_path)

# Show the DataFrame to verify the data
parquet_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [ ]:
# 12. Create a temporary table for the parquet data.

# Read the Parquet formatted data
parquet_path = "/path/to/output_directory"  # Change this to the path where your Parquet files are saved
parquet_df = spark.read.parquet(parquet_path)

# Create a temporary view named 'parquet_home_sales'
parquet_df.createOrReplaceTempView("parquet_home_sales")

# Verify that the temporary view is created
spark.sql("SELECT * FROM parquet_home_sales").show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [ ]:
# Define the query to calculate the average price of a home per "view" rating
from pyspark.sql.functions import avg, round
import time

# Run the query on the Parquet data and measure the runtime
start_time = time.time()
avg_price_by_view_parquet_df = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM parquet_home_sales
    GROUP BY view
    HAVING average_price >= 350000
    ORDER BY view DESC
""")
avg_price_by_view_parquet_df.show()
end_time = time.time()
runtime_parquet = end_time - start_time
print(f"Query run time on Parquet data: {runtime_parquet} seconds")

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1066376.78|
|  98|   1055456.38|
|  97|   1147855.46|
|  96|   1036128.97|
|  95|   1049508.07|
|  94|   1035582.85|
|  93|    973844.22|
|  92|    970908.32|
|  91|   1165257.92|
|  90|   1091155.34|
|  89|   1103479.28|
|  88|   1032742.45|
|  87|   1070579.97|
|  86|   1038134.61|
|  85|   1048783.52|
|  84|   1113276.12|
|  83|   1037127.81|
|  82|   1057533.31|
|  81|   1041586.63|
|  80|    985714.65|
+----+-------------+
only showing top 20 rows

Query run time on Parquet data: 0.622016191482544 seconds


In [ ]:
# Uncache the temporary table
spark.catalog.uncacheTable("home_sales")

# Verify that the table is uncached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")


Is 'home_sales' cached? False
